Azure ML & Azure Databricks notebooks by Parashar Shah.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Register Azure Databricks trained model and deploy it to ACI


Please ensure you have run all previous notebooks in sequence before running this.

Please Register Azure Container Instance(ACI) using Azure Portal: https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-manager-supported-services#portal in your subscription before using the SDK to deploy your ML model to ACI.

In [ ]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
# Set auth to be used by workspace related APIs.
# For automation or CI/CD ServicePrincipalAuthentication can be used.
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.authentication.serviceprincipalauthentication?view=azure-ml-py
auth = None

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config(auth = auth)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
##NOTE: service deployment always gets the model from the current working dir.
import os

model_name = "AdultCensus_runHistory.mml" # 
model_name_dbfs = os.path.join("/dbfs", model_name)

print("copy model from dbfs to local")
model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(model_name, model_local, True)

In [ ]:
#Register the model
from azureml.core.model import Model
mymodel = Model.register(model_path = model_name, # this points to a local file
                       model_name = model_name, # this is the name the model is registered as, am using same name for both path and name.                 
                       description = "ADB trained model by Parashar",
                       workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

In [ ]:
#%%writefile score_sparkml.py
score_sparkml = """
 
import json
 
def init():
    # One-time initialization of PySpark and predictive model
    import pyspark
    import os
    from azureml.core.model import Model
    from pyspark.ml import PipelineModel
 
    global trainedModel
    global spark
 
    spark = pyspark.sql.SparkSession.builder.appName("ADB and AML notebook by Parashar").getOrCreate()
    model_name = "{model_name}" #interpolated
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_name)
    trainedModel = PipelineModel.load(model_path)
    
def run(input_json):
    if isinstance(trainedModel, Exception):
        return json.dumps({{"trainedModel":str(trainedModel)}})
      
    try:
        sc = spark.sparkContext
        input_list = json.loads(input_json)
        input_rdd = sc.parallelize(input_list)
        input_df = spark.read.json(input_rdd)
    
        # Compute prediction
        prediction = trainedModel.transform(input_df)
        #result = prediction.first().prediction
        predictions = prediction.collect()
 
        #Get each scored result
        preds = [str(x['prediction']) for x in predictions]
        result = ",".join(preds)
        # you can return any data type as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        result = str(e)
        return result
    
""".format(model_name=model_name)
 
exec(score_sparkml)
 
with open("score_sparkml.py", "w") as file:
    file.write(score_sparkml)

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas']) # showing how to add libs as an eg. - not needed for this model.

with open("myenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

In [ ]:
#deploy to ACI
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


myaci_config = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                                  memory_gb = 2, 
                                                  tags = {'name':'Databricks Azure ML ACI'}, 
                                                  description = 'This is for ADB and AML example.')

service_name = 'aciws'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

myenv = Environment.get(ws, name='AzureML-PySpark-MmlSpark-0.15')
# we need to add extra packages to procured environment
# in order to deploy amended environment we need to rename it
myenv.name = 'myenv'
model_dependencies = CondaDependencies('myenv.yml')
for pip_dep in model_dependencies.pip_packages:
    myenv.python.conda_dependencies.add_pip_package(pip_dep)
for conda_dep in model_dependencies.conda_packages:
    myenv.python.conda_dependencies.add_conda_package(conda_dep)
inference_config = InferenceConfig(entry_script='score_sparkml.py', environment=myenv)

myservice = Model.deploy(ws, service_name, [mymodel], inference_config, myaci_config)
myservice.wait_for_deployment(show_output=True)

In [ ]:
help(Webservice)

In [ ]:
#for using the Web HTTP API 
print(myservice.scoring_uri)

In [ ]:
import json

#get the some sample data
test_data_path = "AdultCensusIncomeTest"
test = spark.read.parquet(test_data_path).limit(5)

test_json = json.dumps(test.toJSON().collect())

print(test_json)

In [ ]:
#using data defined above predict if income is >50K (1) or <=50K (0)
myservice.run(input_data=test_json)

In [ ]:
#comment to not delete the web service
myservice.delete()

## Deploying to other types of computes

In order to learn how to deploy to other types of compute targets, such as AKS, please take a look at the set of notebooks in the [deployment](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/deployment) folder.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/azure-databricks/amlsdk/deploy-to-aci-04.png)